# Cross Entropy Benchmarking

*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

**Cross entropy benchmarking (XEB)** collects data from different depths of random quantum circuits to determine the fidelity ($F_{XEB}$) and
**Effective Noise Rate (ENR)** of a quantum device.
Google benchmarking their quantum processor Sycamore by this method proved that they reach the quantum supremacy[1].
This tutorial introduces XEB, covering its theory and implementation on [Baidu Quantum Platform](https://quantum.baidu.com/).


## Theory
### Porter-Thomas distribution

For an $n$-bit fixed bitstring $ x^* \in \{0,1\}^{n} $,
a quantum program will specify the probability of sampling this bitstring, define this probability as $P$.
If we draw a quantum program at random, we will get the probability of the probability of sampling this bitstring is.
Let's call it meta-probability and define it as $f(P)$, the probability distribution of meta-probability of drawing some given bitstring, is specified by the Porter-Thomas distribution:

$$
f(p) = Ne^{-Np},
$$

where $N = 2^n$. We have very good complexity-theoretic evidence to believe that, with the increasing of number of qubits, bitstring outputs generated from random quantum programs are exponentially difficult for classical computers to reproduce.

### Random circuits

The common structure of random circuit in XEB is as follows[2]:

![random-circuit](figures/xeb-random-circuit.png "Fig 1 Random Quantum Circuit")

Experimental implementation of random circuit, where only random single qubit gates are used (blue box) with fixed two-qubit gates (white box). Here we define depth as the number of layers contain single qubit gates and fixed two-qubit gates, as red box show in the Figure. Baidu Quantum Platform has the function that implements the random circuit with arbitrary number of qubits and depths. The default settings of function for single qubit gates is *U3* gates and
*CNOT* for two-qubit gates. User can define their own choice of single qubit and two-qubit gates. Now we use the function as example that illustrates the random circuit.

In [ ]:
import numpy as np
from typing import List
import matplotlib.pyplot as plt

from qcompute_qep.utils.circuit import random_circuit, circuit_to_state, print_circuit
from qcompute_qep.benchmarking.xeb import XEB

In [ ]:
qubits=[0, 1] # qubits list
m = 2 # depths
qp = random_circuit(qubits,m)
print_circuit(qp.circuit)

Now we can generate the PT distribution from random circuits by using this function.


In [ ]:
def porter_distribution_sample(qubits: List[int], m_cycle: int, trials: int):
    n_qubits = len(qubits)
    dimension = 2 ** n_qubits
    # picks a random bitstring as labelled by the integers 1 to 2**n_qubits

    probs_bitstring = []

    # Start sample
    for i in range(trials):
        env = random_circuit(qubits, m_cycle)
        state = circuit_to_state(env, vector=True)
        state = state.reshape(-1)
        bitstring = np.random.choice(dimension)
        prob = np.abs(state[bitstring]) ** 2
        probs_bitstring.append(prob)
    return probs_bitstring


def plot_sample_result(n_qubits, probs_bitstring):
    dim = 2 ** n_qubits
    xspace = np.linspace(0.0, 1.0, 100)
    yspace = dim * np.exp(-dim * xspace)

    # plot both empirical and theoretical calculations
    plt.figure(figsize=(9, 6))
    plt.hist(probs_bitstring, bins=20, density=True, label='Empirical Distribution')
    plt.plot(xspace, yspace, label='Theoretical Porter-Thomas Distribution')

    plt.xlabel("Probability p")
    plt.ylabel("Probability that the random bistring occurs with probability p")
    plt.legend(loc='best')
    plt.show()

qubits = [0, 1]
N = len(qubits)
M = 5  # depths
trials = 1 * 10 ** 4  # number of sample
plot_sample_result(N, porter_distribution_sample(qubits, M, trials))

As shown above, the resulting distribution is in consistent with the theoretical distribution.

Notice tha the following factors will affect the randomness of circuit.

1. structure of random circuit

2. choice of quantum gates

3. number of qubits

4. depths of circuit


### Randomized benchmarking

The fidelity $F_{XEB}$ as mention above is the distance metric between the probability distributions of the outputs from noise circuit and ideal circuit. The example of PT distribution sampling above can be regarded as sampling a maximally mixed stated, since we sample all the possible result with equal probability. We calculate the fidelity by using linear cross entropy. The linear cross entropy is defined as

$$
\mathcal L\left(P\parallel Q\right) = 2^{n}\sum\limits_{x}P(x)Q(x)-1.
$$

To calculate the fidelity, we use the following equation

$$
F_{XEB} = \mathcal L\left(P_{noisy}\parallel P_{ideal}\right) \approx \frac{D}{RS}\sum\limits_{r=1}^{R}\sum\limits_{s=1}^{S}P_{ideal}^{(r)}\left(x_{s}^{(r)}\right)-1,
$$

where $D = 2^n$, $R$ is number of circuits for each depths, $S$ is shots of each circuit, $P_{ideal}^{(r)}$ is the ideal probability, and $ x_{s}^{(r)} $ is the outcome bitstring samples obtained from random circuit on the noisy quantum device. When the samples $x_{s}^{(r)}$ come from the ideal distribution, the $F_{XEB}$ can be exponentially large. So we consider the **unbiased linear cross entropy** $ F_{uXEB} $, defined as:

$$
F_{uXEB} = \frac{\frac{D}{RS}\sum_{r,s=1}^{RS}P_{ideal}^{(r)}\left(x_{s}^{(r)}\right)-1} {\frac{D}{R}\sum_{r=1}^{R}{\left(P_{ideal}^{(r)}\left(x^{(r)}\right)\right)}^{2}-1},
$$

where $x^{r} \in \{0,1\}^{n}$. When the circuit is ideal circuit, the $F_{uXEB} = 1$. After calculating the $F_{uXEB}$ from different depths of circuit. We can fit the data as following equation to get **ENR**:

$$
F = Ae^{-{\lambda}d},
$$

where $d$ is depth, $F$ is $F_{uXEB}$, and $\lambda$ is **ENR**.

## Practice

We can use following codes to obtain the outcomes and plot the result.

### Single qubit circuit

In [ ]:
import QCompute

# Please login the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
QCompute.Define.hubToken = "Token"

qc = QCompute.BackendName.CloudBaiduQPUQian
xeb_1 = XEB()
xeb_1_results = xeb_1.benchmark(qubits=[0],
                            qc=qc,
                            shots=8192,
                            repeats=20,
                            seq_lengths=[1, 5, 10, 15, 20, 30, 40])
xeb_1.plot_results()


### Two qubit circuit

In [ ]:
xeb_2 = XEB()
xeb_2_results = xeb_2.benchmark(qubits=[0, 1 ],
                            qc=qc,
                            shots=8192,
                            repeats=20,
                            seq_lengths=[1, 5, 10, 15, 20, 30, 40])
xeb_2.plot_results()

We get the fitting curve by the sampling rom different depth of random circuit.
Let's print the result to get the ENR.

In [ ]:
print("The ENR is: ",xeb_1_results['lambda'])
print("The ENR is: ",xeb_2_results['lambda'])

We have finished the XEB and successfully obtained the **ENR** of noisy quantum device. User can define different noise model and parameters to obtain more results. More details can find on this paper[2].

## References

\[1\] Arute, F., Arya, K., Babbush, R. et al. Quantum supremacy using a programmable superconducting processor. [Nature](https://www.nature.com/articles/s41586-019-1666-5) 574, 505–510 (2019).

\[2\] Liu, Yunchao, et al. "Benchmarking near-term quantum computers via random circuit sampling." [arXiv](https://arxiv.org/abs/2105.05232) preprint arXiv:2105.05232 (2022).